# Generating Asteroids

In [39]:
// Let's visualize our functions via a helper method
using XPlot.Plotly;
using System.Numerics;
public XPlot.Plotly.PlotlyChart LinearPlotChart(Func<float, float> function, string title, string xaxis, string yaxis)
{
    var data = new Graph.Scatter()
    {    
        x = Enumerable.Range(-200, 400).Select(xv => xv / 100.0f),
        y = Enumerable.Range(-200, 400).Select(yv => function(yv / 100.0f))        
    };
    
    return Plot(data, -2.0f, 2.0f, -2.0f, 2.0f, title, xaxis, yaxis);
}

public XPlot.Plotly.PlotlyChart CirclePlotChart(Func<float, float> function, string title, string xaxis, string yaxis)
{    
    var craterPosition = new Vector2(0, 1);
    Func<float, float> distance = f => Vector2.Distance(new Vector2((float)Math.Cos(f), (float)Math.Sin(f)), craterPosition);
        
    var offset = 0.0f;//Math.PI / 2;
    var xs = Enumerable.Range(0, 628).Select(v => v / 100.0f).Select(v => (float)Math.Cos(v) * function(distance(v)));
    var ys = Enumerable.Range(0, 628).Select(v => v / 100.0f).Select(v => (float)Math.Sin(v) * function(distance(v)));
    var ds = Enumerable.Range(0, 628).Select(v => v / 100.0f).Select(v => distance(v));
                
    var data = new Graph.Scatter() {x = xs, y = ys, mode = "markers", marker = new Graph.Marker() { color = ds } };
       
    return Plot(data, -1.1f, 1.1f, -1.1f, 1.1f, title, xaxis, yaxis);
}

public XPlot.Plotly.PlotlyChart Plot(Graph.Scatter data, float minX, float maxX, float minY, float maxY, string title, string xaxis, string yaxis)
{
     var layout = new Layout.Layout();
    layout.xaxis = new Graph.Xaxis() { range = new float[] { minX, maxX } };
    layout.yaxis = new Graph.Yaxis() { range = new float[] { minY, maxY } };

    var chart = Chart.Plot(data, layout);
    chart.WithTitle(title);    
    chart.WithXTitle(xaxis);
    chart.WithYTitle(yaxis);
    chart.WithWidth(750);
    chart.WithHeight(750);
    
    return chart;
}

var craterRadius = 1.0f;
var rimWidth = 0.0f;

Func<float, float> distance = d => d / Math.Max(craterRadius, 0.0001f);
Func<float, float> cavity = x => distance(x) * distance(x) - 1;
Func<float, float> combined = x => (Math.Min(cavity(x), 1.0f) + 1.0f) / 2.0f;

display(CirclePlotChart(combined, "Asteroid", "Distance to Crater Center", "Crater Height"));
display(LinearPlotChart(combined, "Function", "Distance", "Height"));
